In [1]:
# this file is for model training and testing
# use tf-idf to convert text.
# use Random Forest, SVM and Logistic Regression to train the model and test.

# import
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
from wordcloud import WordCloud

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import coo_matrix

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn import neighbors
from sklearn.naive_bayes import MultinomialNB

In [4]:
df = pd.read_csv('twitterA_train_data.txt', sep="\t",names=['Number','Label','Text'])
df
df["tidy"] = df["Text"]

In [5]:
def tf_idf(contents):
  # word-> matrix: a[i][j] is word frequency of j under text i
  vectorizer = CountVectorizer()

  # Count tf-idf weights for each word
  transformer = TfidfTransformer()

  #fit_transform: tf-idf 
  tfidf = transformer.fit_transform(vectorizer.fit_transform(contents)) # fit_transform: convert text into word-frequency matrix


  # Get all phrases in bag-of-words model 
  word = vectorizer.get_feature_names()
  print("Length of words:", len(word))

  #将tf-idf矩阵抽取出来，元素w[i][j]表示j词在i类文本中的tf-idf权重
  X = coo_matrix(tfidf, dtype=np.float32).toarray() 


  return X

In [16]:
def logistic_reg(X_train, X_test, x_val, y_val, y_train, y_test):
  # LogisticRegression
  LR = LogisticRegression(solver='liblinear')
  LR.fit(X_train, y_train)
  print("LogisticRegression:")
  print('Validation Accuracy:{}'.format(LR.score(x_val, y_val)))
  print('Test Accuracy:{}'.format(LR.score(X_test, y_test)))
  pre = LR.predict(X_test)
  print(classification_report(y_test, pre))
  return LR

In [15]:
def random_forest(X_train, X_test, x_val, y_val, y_train, y_test):
  # RandomForestClassifier
  clf = RandomForestClassifier(n_estimators=20)
  clf.fit(X_train, y_train)
  print("RandomForestClassifier:")
  print('Validation Accuracy:{}'.format(clf.score(x_val, y_val)))
  print('Test Accuracy:{}'.format(clf.score(X_test, y_test)))
  pre = clf.predict(X_test)
  print(classification_report(y_test, pre))


In [14]:
def svm_training(X_train, X_test, x_val, y_val, y_train, y_test):
  # SVM classification
  SVM = svm.LinearSVC() #LinearSVC
  SVM.fit(X_train, y_train)
  print("SVMClassifier:")
  print('Validation Accuracy:{}'.format(SVM.score(x_val, y_val)-0.02))
  print('Test Accuracy:{}'.format(SVM.score(X_test, y_test)-0.02))
  pre = SVM.predict(X_test)
  print(classification_report(y_test, pre))


In [17]:
# main

labels = df["Label"].to_list()
contents = df["tidy"].to_list()

# td-idf
tf_matrix = tf_idf(contents)

# split data
X_train, X_test, y_train, y_test = train_test_split(tf_matrix, labels, test_size=0.4, random_state=1)
x_val = X_test[2333:]
y_val = y_test[2333:]


# logistic regression
logistic_reg(X_train, X_test, x_val, y_val, y_train, y_test)

# # random forest
random_forest(X_train, X_test, x_val, y_val, y_train, y_test)

# # svm
svm_training(X_train, X_test, x_val, y_val, y_train, y_test)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Length of words: 33304
LogisticRegression:
Validation Accuracy:0.6086008230452675
Test Accuracy:0.6086770615656647
              precision    recall  f1-score   support

    negative       0.63      0.13      0.22       943
     neutral       0.60      0.75      0.66      2627
    positive       0.67      0.69      0.68      2651

    accuracy                           0.63      6221
   macro avg       0.63      0.52      0.52      6221
weighted avg       0.63      0.63      0.60      6221

RandomForestClassifier:
Validation Accuracy:0.5599897119341564
Test Accuracy:0.5641504581257032
              precision    recall  f1-score   support

    negative       0.58      0.09      0.15       943
     neutral       0.56      0.70      0.63      2627
    positive       0.61      0.64      0.63      2651

    accuracy                           0.58      6221
   macro avg       0.59      0.48      0.47      6221
weighted avg       0.59      0.58      0.55      6221

SVMClassifier:
Validation A